This script will allow you to impedance match the measured shock velocity of a transparent sample with an aluminum flyer. The Al data were taken from Knudson et al. JAP 2003 (DOI: 10.1063/1.1604967) and used in Root et al. GRL 2018 (DOI: 10.1029/2017GL076931).

Inputs:
flyer V is the velocity of the Al flyer plate
flyer V_e is the error associated with Al flyer plate velocity
Us_s is the shock velocity in the sample measured with VISAR, Us_s_e is the error in shock velocity
rho_s is the initial density of the sample material, rho_s_e is the error in initial density

In [ ]:
##Single sample
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as sp
from scipy import linalg
from scipy.optimize import fsolve

#Some uncertainties are assumed

# flyer V is the velocity of the Al flyer plate
# flyer V_e is the error associated with Al flyer plate velocity
# Us_s is the shock velocity in the sample measured with VISAR, Us_s_e is the error in shock velocity
# rho_s is the initial density of the sample material

def IM(flyerV,flyerV_e,Us_s,Us_s_e,rho_s,rho_s_e):

    # 1. Parameter read in
    Covparams = [[0.000419554462407,  -0.004605429541366],
                  [-0.004605429541366,   0.053580461575632]] ## Al covariance matrix
    
    frho = 2700 #kg/m^3, density of the Al flyer
    frho_e = frho*0.03 # uncertainty

    steps = 10000 #as many steps as is fast, more for more accuracy.

    #initialize arrays
    lmat = sp.linalg.cholesky(Covparams,lower=True)
    root = np.ones(steps)
    upmc = np.ones(steps)
    rhomc = np.ones(steps)
    Pmc = np.ones(steps)

    i=0
    while i < steps :
        flyerv = flyerV + flyerV_e * sp.randn()
        flyrho = frho + frho_e * sp.randn()
        samus = Us_s + Us_s_e * sp.randn()
        samrho = rho_s + rho_s_e * sp.randn()

        temp_mat=sp.randn(2,2)
        bmat=np.matmul(lmat,temp_mat)
        S1 = 1.188545520433363 + bmat[0,0]
        C0 = 6.322038234532245 + bmat[0,1]

        def bigP(up,S1,C0,flyrho,flyerv,samrho,samus):
            return flyrho*(S1 * up**2 - (C0 + 2 * S1 * flyerv) * up + C0 * flyerv + S1 * flyerv**2) - \
                    (samrho * samus * up)

        root = fsolve(bigP,5,args=(S1,C0,flyrho,flyerv,samrho,samus))

        upmc[i] = root
        rhomc[i] = (samus/(samus-upmc[i]))*samrho
        Pmc[i] = samrho*samus*upmc[i]

        i = i+1

    up = np.mean(upmc)
    uperr = np.std(upmc)
    P = np.mean(Pmc)*1e-3
    Perr = np.std(Pmc)*1e-3
    rho = np.mean(rhomc)*1e-3
    rhoerr = np.std(rhomc)*1e-3

    print('up=',up,uperr)
    print('P=',P,Perr)
    print('rho=',rho,rhoerr)